In [ ]:
#find terms which have no code from table medications
import sqlite3

conn=sqlite3.connect("/Users/dc/TestCode/TestCoding/clinical.sqlite")
c = conn.cursor()
c.execute("select count(code) from medications where code_system_name=\"rxnorm\" and code not in(select DISTINCT(RXCUI) from rxnconso)")
print c.fetchone()

f=open("/Users/dc/TestCode/TestCoding/nomatch.csv", "r")
queries=[]
for readline in f:
    queries.append(readline)
    
print len(queries)
for i in range(10):
    print queries[i]

In [ ]:
import sqlite3
import logging

class MyDB(object):
    _db_connection = None
    _db_cur = None
    db = "/Users/dc/TestCode/TestCoding/clinical.sqlite"
    
    def __init__(self):
        self._db_connection = sqlite3.connect(db)
        self._db_cur = self._db_connection.cursor()

    def query(self, query, params):
        return self._db_cur.execute(query, params)

    def __del__(self):
        self._db_connection.close()


In [3]:

import sqlite3
import re
import logging

class Match(object):
    conn = sqlite3.connect('/Users/dc/TestCode/TestCoding/clinical.sqlite')
    c=conn.cursor()
    
    def __init__(self):
        logger = logging.getLogger(__name__)
        handler = logging.FileHandler(filename='/Users/dc/TestCode/TestCoding/match.log', mode='w')
        handler.setLevel(logging.INFO)
        logger.addHandler(handler)
        logger.info("match init")        
        self.baseResult = []
        self.ParenResult = []
        self.matchResult= []
        
    def processQuery(self,query):
        findParen = query.find("(")
        if findParen != -1:
            self.processBase(query[0:findParen])
            print "found paren numBaseResult:%d" % len(self.baseResult) 
            logger.info( "found paren numBaseResult:%d" % len(self.baseResult)) 
            self.processParen(query[findParen+1:query.find(")")])
        else:
            self.baseResult = self.processBase(query)
            
        
    def processBase(self,query):
        cleanQ = self.cleanBaseQuery(query)
        print "processing query:%s" % cleanQ
        logger.info("processing query:%s" % cleanQ)
        q = "select * from lookup where name like \'%"+cleanQ+"%\'"
        print "processBase q:%s" % q
        logger.info("processBase q:%s" % q)
        self.c.execute(q)
        self.baseResult = self.c.fetchall()
        print "num baseResult:" , len(self.baseResult)
        print "setBaseResult:", len(set(self.baseResult))
        
        
        qDistinct = "select distinct(rxcui) from lookup where name like \'%"+cleanQ+"%\'"
        print "distinctQuery:%s" % qDistinct
        self.c.execute(qDistinct)
        distinctResult = self.c.fetchall()
        print "numDistinct:", len(distinctResult)
        print "set numDistinct:", len(set(distinctResult))
        
        
    def processParen(self,query):
        """
        using ferrous sulfate example the paren terms are a separate set and have to
        be anded with the original terms. Return list with best match and put on top. 
        """
        print "processParen query:%s" % query
        terms = self.cleanQuery(query).split()
        print "processParen terms:%s" % terms
        poExists=False
        if "po" in terms:
            terms.remove("po")
            poExists = True
        
        qTerms = " ".join(terms)
        qString = "select * from lookup where name like '%"+qTerms+"%'"
        print "processParen qString:%s" % qString
        self.c.execute(qString)
        self.parenResult=self.c.fetchall()
        print "numResults self.parenResult:%d" % len(self.parenResult)
        print "numDistinct self.parenResult:%d" % len(set(self.parenResult))
        distinctString = "select distinct(rxcui) from lookup where name like '%"+qTerms+"%'"
        self.c.execute(distinctString)
        distinctResult = self.c.fetchall()
        print "numDistinct:%d" % len(distinctResult)
        print "set numDistinct:%d" % len(set(distinctResult))
        
        for r in set(self.parenResult):
            print r
        for r in set(self.parenResult):
            if poExists:
                terms.append("oral")
            print "single parenResult: " , r , "numTuples:", len(r), "name tokens:", r[0].split(), "set r[0]:" , set(r[0].split())
            print "set terms:" , set(terms)
            print set(terms).issubset(set(r[0].split()))
        
    
    def cleanBaseQuery(self, query):
        """
        """
        return query.lower().strip()
                                      
    def cleanQuery(self,query):
        """
        this will end up becoming list of queries PO->oral, pill, etc...
        """
        print "query before clean punct:%s" % query
        cleanPunct = re.sub("-"," ",query.lower())
        print "clean punct:%s" % cleanPunct
        return cleanPunct
    
    def test(self):
        with open("/Users/dc/TestCode/TestCoding/nomatch.csv") as f:
            for i in range(5):
                line  = f.readline()
                print line
                self.processQuery(line)
            
m = Match()
m.test()


INFO:__main__:match


name:__main__
Ferrous Sulfate (SLOW FE PO)|null

processing query:ferrous sulfate
processBase q:select * from lookup where name like '%ferrous sulfate%'
num baseResult: 1039
setBaseResult: 872
distinctQuery:select distinct(rxcui) from lookup where name like '%ferrous sulfate%'
numDistinct: 473
set numDistinct: 473
found paren numBaseResult:1039
processParen query:SLOW FE PO
query before clean punct:SLOW FE PO
clean punct:slow fe po
processParen terms:['slow', 'fe', 'po']
processParen qString:select * from lookup where name like '%slow fe%'
numResults self.parenResult:27
numDistinct self.parenResult:22
numDistinct:10
set numDistinct:10
(u'ferrous sulfate 160 mg / folic acid 0.4 mg [slow fe with folic acid]', u'572935')
(u'slow fe, 160 mg oral tablet, extended release', u'210095')
(u'ferrous sulfate/folic acid 50 mg-0.4 mg oral tablet, extended release [slow fe with folic acid]', u'212973')
(u'ferrous sulfate 160 mg / folic acid 0.4 mg extended release oral tablet [slow fe with folic aci